# Table of Contents
 <p><div class="lev1"><a href="#Task-1.-Scraping-Top-Universities"><span class="toc-item-num">Task 1.&nbsp;&nbsp;</span>Scraping Top Universities</a></div>
 <div class="lev1"><a href="#Task-2.-Scraping-Times-Higher-Education"><span class="toc-item-num">Task 2.&nbsp;&nbsp;</span>Scraping Times Higher Education</a></div>
 <div class="lev1"><a href="#Task-3.-Merging-Both-Rankings"><span class="toc-item-num">Task 3.&nbsp;&nbsp;</span>Merging Both Rankings</a></div></p>

## Task 1. Scraping Top Universities

### Assignment Instructions
Obtain the 200 top-ranking universities in www.topuniversities.com ([ranking 2018](https://www.topuniversities.com/university-rankings/world-university-rankings/2018)). In particular, extract the following fields for each university: name, rank, country and region, number of faculty members (international and total) and number of students (international and total). Some information is not available in the main list and you have to find them in the [details page](https://www.topuniversities.com/universities/ecole-polytechnique-fédérale-de-lausanne-epfl).
Store the resulting dataset in a pandas DataFrame and answer the following questions:
- Which are the best universities in term of: (a) ratio between faculty members and students, (b) ratio of international students?
- Answer the previous question aggregating the data by (c) country and (d) region.

Plot your data using bar charts and describe briefly what you observed.

### 1.1. Scraping the 200 top elements from Top Universities

We import some libraries.

In [1]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
from pandas.io.json import json_normalize

We establish constants we need for scraping. We found the *json* URL by using Postman on the URL given in the instructions.

In [2]:
TOP_UNIVERSITIES_URL = 'https://www.topuniversities.com'
TOP_UNIVERSITIES_JSON_URL = TOP_UNIVERSITIES_URL + '/sites/default/files/qs-rankings-data/357051.txt'
TOP_UNIVERSITIES_JSON_COLUMNS = ['title', 'rank_display', 'country', 'region']
TOP_UNIVERSITIES_HTML_COLUMNS = ['total faculty', 'inter faculty', 'total student', 'total inter']
TOP_UNIVERSITIES_NEW_COLUMNS = ['Name', 'Rank', 'Country', 'Region', 'Faculty', 'International Faculty', 'Students', 'International Students']

We will use the following helper functions for scraping:
- `get_json` fetches the *json* data for a given URL;
- `get_html_beautiful_soup` gets the `BeautifulSoup` associated with a URL;
- `get_number` gets the `int` associated with a given class name in a given `BeautifulSoup` instance.

In [3]:
def get_json(url):
    """
    Returns the json associated with given url, or an empty dict if an error arises.
    :param url: string, url target
    :return: dict
    """
    try:
        return requests.get(url).json()
    except:
        return {}

def get_html_beautiful_soup(row, url_prefix):
    """
    Returns BeautifulSoup of the target URL, or None if an error arises.
    :param row: Pandas Series, corresponding to a row of a DataFrame.
    :param url_prefix: string, corresponding to URL prefix
    :return: BeautifulSoup, or None
    """
    try:
        return BeautifulSoup(requests.get(url_prefix + row['url']).text, 'html.parser')
    except:
        return None

def get_number(soup, class_name):
    """
    Returns number associated with the given class name in the soup passed as argument, or NaN if an error arises.
    :param soup: BeautifulSoup, extracted beforehand
    :param class_name: string, targeted class name
    :return: int, or NaN
    """
    try:
        number_text = soup.find('div', class_= class_name).find('div', class_='number').text
        return int(''.join([char for char in number_text if char.isdigit()]))
    except:
        return np.NaN

We use the following function to scrape universities.

In [4]:
def scrape_universities(json_url, current_columns, new_columns, extra_columns=[], extra_url='', max_universities=200):
    """
    Returns a DataFrame instance containing all university data.
    :param json_url: string, url containing json file
    :param current_columns: list of strings, containing columns to keep
    :param new_columns: list of strings, same length as current_columns, to rename the DataFrame's columns
    :param extra_columns: list of strings, extra columns to get from specific university pages
    :param extra_url: string, url prefix to get to specific university pages
    :param max_universities: int, number of top universities to get, default is 200
    :return: DataFrame
    """
    university_df = json_normalize(get_json(json_url)['data'][:max_universities])
    if extra_columns:
        beautiful_soups = university_df.apply(lambda row: get_html_beautiful_soup(row, extra_url), axis=1)
        for column in extra_columns:
            university_df[column] = beautiful_soups.apply(lambda soup: get_number(soup, column))
    university_df = university_df[current_columns].rename(index=str, columns=dict(zip(current_columns, new_columns)))
    return university_df

We scrape the top 200 universities.

In [42]:
top_universities_df = scrape_universities(TOP_UNIVERSITIES_JSON_URL,
                                          TOP_UNIVERSITIES_JSON_COLUMNS + TOP_UNIVERSITIES_HTML_COLUMNS,
                                          TOP_UNIVERSITIES_NEW_COLUMNS, 
                                          extra_columns=TOP_UNIVERSITIES_HTML_COLUMNS,
                                          extra_url=TOP_UNIVERSITIES_URL)
top_universities_df.head()

,Name,Rank,Country,Region,Faculty,International Faculty,Students,International Students
0,Massachusetts Institute of Technology (MIT),1,United States,North America,2982.0,1679.0,11067.0,3717.0
1,Stanford University,2,United States,North America,4285.0,2042.0,15878.0,3611.0
2,Harvard University,3,United States,North America,4350.0,1311.0,22429.0,5266.0
3,California Institute of Technology (Caltech),4,United States,North America,953.0,350.0,2255.0,647.0
4,University of Cambridge,5,United Kingdom,Europe,5490.0,2278.0,18770.0,6699.0


### 1.2. Sorting by ratio between faculty members and students

We establish a sorting helper function, called `insert_column_and_sort`, to insert and compute a new column and then sort the values by this new column.

In [43]:
def insert_column_and_sort(dataframe, columns, head_elements=5):
    """
    Computes new column based on the division of one column by another one and returns it.
    :param dataframe: DataFrame, targeted instance
    :param columns: list of 3 strings and 2 booleans, containing names of the new column, 
                    numerator column and denominator column, and 2 booleans determining
                    if denominator column includes numerator one, and order of sorting
    :param head_elements: int, number of head elements to be shown
    :return: DataFrame
    """
    df_copy = dataframe.copy()
    new_column, numerator, denominator, denominator_includes_numerator, ascending = columns
    numerator_df = dataframe[numerator]
    denominator_df = dataframe[denominator]
    if denominator_includes_numerator:
        denominator_df = denominator_df - numerator_df
    df_copy[new_column] = numerator_df / denominator_df
    return df_copy.sort_values(new_column, ascending=ascending).head(head_elements)

We introduce a new column for the ratio between faculty members and students, such that each value is the average number of students per faculty member. Then, we sort the top universities scraped by this ratio in ascending order.

In [44]:
STUDENT_STAFF_RATIO = ['Student-Staff Ratio', 'Students', 'Faculty', False, True]
insert_column_and_sort(top_universities_df, STUDENT_STAFF_RATIO)

,Name,Rank,Country,Region,Faculty,International Faculty,Students,International Students,Student-Staff Ratio
3,California Institute of Technology (Caltech),4,United States,North America,953.0,350.0,2255.0,647.0,2.366212
15,Yale University,16,United States,North America,4940.0,1708.0,12402.0,2469.0,2.510526
5,University of Oxford,6,United Kingdom,Europe,6750.0,2964.0,19720.0,7353.0,2.921481
4,University of Cambridge,5,United Kingdom,Europe,5490.0,2278.0,18770.0,6699.0,3.418944
16,Johns Hopkins University,17,United States,North America,4462.0,1061.0,16146.0,4105.0,3.618557


### 1.3. Sorting by ratio of international students

Likewise, we introduce a new column for the ratio of international students, such that each value is the average number of international students per non-international student. Then, we sort the top universities scraped by this ratio in **descending** order.

In [45]:
INTL_STUDENT_RATIO = ['International Student Ratio', 'International Students', 'Students', True, False]
insert_column_and_sort(top_universities_df, INTL_STUDENT_RATIO)

,Name,Rank,Country,Region,Faculty,International Faculty,Students,International Students,International Student Ratio
34,London School of Economics and Political Scien...,35,United Kingdom,Europe,1088.0,687.0,9760.0,6748.0,2.240372
11,Ecole Polytechnique Fédérale de Lausanne (EPFL),12,Switzerland,Europe,1695.0,1300.0,10343.0,5896.0,1.325838
7,Imperial College London,8,United Kingdom,Europe,3930.0,2071.0,16090.0,8746.0,1.190904
198,Maastricht University,200,Netherlands,Europe,1277.0,502.0,16385.0,8234.0,1.010183
47,Carnegie Mellon University,=47,United States,North America,1342.0,425.0,13356.0,6385.0,0.915937


### 1.4. Sorting while grouped by country

We make a short helper function, called `group_insert_sort`, to group by a column, insert another one and sort by its values.

In [46]:
def group_insert_sort(dataframe, group_by, columns, head_elements=5):
    """
    Computes new column based on the division of one column by another one and returns it.
    :param dataframe: DataFrame, targeted instance
    :param group_by: string, name of the column by which dataframe is grouped
    :param columns: list of 3 strings and 2 booleans, containing names of the new column, 
                    numerator column and denominator column, and 2 booleans determining
                    if denominator column includes numerator one, and order of sorting
    :param head_elements: int, number of head elements to be shown
    :return: DataFrame
    """
    return insert_column_and_sort(dataframe.groupby(group_by).sum(), columns, head_elements)

The top universities by country and by student-staff ratio are the following:

In [47]:
group_insert_sort(top_universities_df, 'Country', STUDENT_STAFF_RATIO)

,Faculty,International Faculty,Students,International Students,Student-Staff Ratio
Country,,,,,
Russia,6709.0,373.0,30233.0,5098.0,4.506335
Denmark,11916.0,3904.0,67223.0,9543.0,5.641407
Saudi Arabia,1062.0,665.0,6040.0,989.0,5.687382
Singapore,9444.0,6079.0,58466.0,16168.0,6.190809
Malaysia,2755.0,655.0,17902.0,3476.0,6.498004


The top universities by country and by international student ratio are the following:

In [48]:
group_insert_sort(top_universities_df, 'Country', INTL_STUDENT_RATIO)

,Faculty,International Faculty,Students,International Students,International Student Ratio
Country,,,,,
Australia,22034.0,11382.0,301994.0,106359.0,0.543660
United Kingdom,79934.0,30216.0,583621.0,199426.0,0.519075
Hong Kong,10166.0,6296.0,78838.0,24499.0,0.450855
Austria,4117.0,1572.0,63446.0,19667.0,0.449234
Switzerland,15323.0,9208.0,109112.0,32995.0,0.433477


### 1.5. Sorting while grouped by region

The top universities by region and by student-staff ratio are the following:

In [49]:
group_insert_sort(top_universities_df, 'Region', STUDENT_STAFF_RATIO)

,Faculty,International Faculty,Students,International Students,Student-Staff Ratio
Region,,,,,
Asia,106734.0,25462.0,807003.0,110100.0,7.560880
North America,182123.0,43836.0,1546353.0,292116.0,8.490707
Europe,218358.0,67598.0,1957251.0,449364.0,8.963496
Latin America,45382.0,5648.0,435750.0,36871.0,9.601825
Africa,1733.0,379.0,19593.0,3325.0,11.305828


The top universities by region and by international student ratio are the following:

In [50]:
group_insert_sort(top_universities_df, 'Region', INTL_STUDENT_RATIO)

,Faculty,International Faculty,Students,International Students,International Student Ratio
Region,,,,,
Oceania,25347.0,12786.0,350167.0,118798.0,0.513457
Europe,218358.0,67598.0,1957251.0,449364.0,0.298009
North America,182123.0,43836.0,1546353.0,292116.0,0.232903
Africa,1733.0,379.0,19593.0,3325.0,0.204389
Asia,106734.0,25462.0,807003.0,110100.0,0.157985


## Task 2. Scraping Times Higher Education

### Assignment Instructions

Obtain the 200 top-ranking universities in www.timeshighereducation.com ([ranking 2018](http://timeshighereducation.com/world-university-rankings/2018/world-ranking)). Repeat the analysis of the previous point and discuss briefly what you observed.

### 2.1. Scraping the top 200 elements from Times Higher Education

We establish the following constants.

In [51]:
TIMES_HIGHER_ED_JSON_URL = 'https://www.timeshighereducation.com/sites/default/files/the_data_rankings/world_university_rankings_2018_limit0_369a9045a203e176392b9fb8f8c1cb2a.json'
TIMES_HIGHER_ED_JSON_COLUMNS = ['name', 'rank', 'location', 'stats_number_students', 'stats_student_staff_ratio', 'stats_pc_intl_students', 'stats_female_male_ratio']
TIMES_HIGHER_ED_NEW_JSON_COLUMNS = ['Name', 'Rank', 'Country', 'Students', 'Student-Staff Ratio', 'Percentage of International Students', 'Female-Male Ratio']

Then we scrape the universities from Times Higher Education.

In [52]:
times_higher_ed_df = scrape_universities(TIMES_HIGHER_ED_JSON_URL, 
                                         TIMES_HIGHER_ED_JSON_COLUMNS, 
                                         TIMES_HIGHER_ED_NEW_JSON_COLUMNS)
times_higher_ed_df.head()

,Name,Rank,Country,Students,Student-Staff Ratio,Percentage of International Students,Female-Male Ratio
0,University of Oxford,1,United Kingdom,"20,409",11.2,38%,46 : 54
1,University of Cambridge,2,United Kingdom,"18,389",10.9,35%,45 : 55
2,California Institute of Technology,=3,United States,"2,209",6.5,27%,31 : 69
3,Stanford University,=3,United States,"15,845",7.5,22%,42 : 58
4,Massachusetts Institute of Technology,5,United States,"11,177",8.7,34%,37 : 63


In [ ]:
# TODO
# Compute sorting stuff

## Task 3. Merging Both Rankings

### Assignment Instructions

Merge the two DataFrames created in questions 1 and 2 using university names. Match universities' names as well as you can, and explain your strategy. Keep track of the original position in both rankings.

### Answers

We merge both tables that we scraped in tasks 1 and 2.

We first import a library and make a short function to find the closest match in names.

In [53]:
import difflib

def find_closest_match(name, other_names):
    """
    Returns the closest match to given name in a series of other names, or just the name if nothing is found.
    :param name: string, given name that we want to find
    :param other_names: Pandas Series, names that the argument name will be compared to
    :return: string
    """
    try:
        return difflib.get_close_matches(name, other_names)[0]
    except:
        return name

We replace the column `'Name'` in the Times Higher Education `DataFrame` by its closest match in the Top Universities one.

In [54]:
NAME = 'Name'
times_higher_ed_df[NAME] = times_higher_ed_df[NAME].apply(lambda name: find_closest_match(name, top_universities_df[NAME]))

Then we merge both `DataFrame` instances into one.

In [55]:
merged_df = pd.merge(top_universities_df, times_higher_ed_df, how='outer', on='Name', suffixes=(' (Top Unis)', ' (T.H.E.)'))
merged_df.head()

,Name,Rank (Top Unis),Country (Top Unis),Region,Faculty,International Faculty,Students (Top Unis),International Students,Rank (T.H.E.),Country (T.H.E.),Students (T.H.E.),Student-Staff Ratio,Percentage of International Students,Female-Male Ratio
0,Massachusetts Institute of Technology (MIT),1,United States,North America,2982.0,1679.0,11067.0,3717.0,5,United States,"11,177",8.7,34%,37 : 63
1,Stanford University,2,United States,North America,4285.0,2042.0,15878.0,3611.0,=3,United States,"15,845",7.5,22%,42 : 58
2,Harvard University,3,United States,North America,4350.0,1311.0,22429.0,5266.0,6,United States,"20,326",8.9,26%,None
3,California Institute of Technology (Caltech),4,United States,North America,953.0,350.0,2255.0,647.0,=3,United States,"2,209",6.5,27%,31 : 69
4,University of Cambridge,5,United Kingdom,Europe,5490.0,2278.0,18770.0,6699.0,2,United Kingdom,"18,389",10.9,35%,45 : 55


In [ ]:
# TODO
# Remove duplicate columns

## Task 4. Correlation

See Pearson's correlation, Rank correlation, e.g., Spearman’s correlation coefficient, mutual information

Check for Simpson's paradox, test some hypotheses with **p-values**.